In [141]:
import os
import sys
from time import gmtime, strftime

from keras.models import load_model
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import pandas as pd
import math
import keras.backend as K
from scipy import sparse
from scipy.sparse import vstack

sys.path.append('../../data')
sys.path.append('../../src/models')
from recommenders.cf_recommender import CFRecommender

k = 5 #int(sys.argv[1])
autoencoder_model = 'train_autoencoder_128_cdae_users_projects' #str(sys.argv[2]) # 'train_autoencoder_32_cdae_users_projects'
dataSource = 'users_projects' # str(sys.argv[3]) # 'movies' 

# Load the autoencoder to use
model = load_model('../../data/autoencoders/' + autoencoder_model + '.h5')
projects = pd.read_pickle('../../data/processed/project_data')

In [142]:
def load_users_projects():
    cf = pd.read_pickle('../../data/processed/cf_projects.pkl')
    #cf = pd.read_pickle('data/processed/cf_profiles.pkl')

    train_x = sparse.load_npz("../../data/processed/train_sparse.npz")
    val_x = sparse.load_npz("../../data/processed/val_sparse.npz")
    test_x = sparse.load_npz("../../data/processed/test_sparse.npz")
    
    train_labels = cf
    val_labels = cf
    test_labels = cf

    return train_labels, train_x, val_labels, val_x, test_labels, test_x

def load_profile_labels():
    cf_profiles = pd.read_pickle('../../data/processed/cf_profiles.pkl')

    return cf_profiles

In [143]:
# Load out time consistent collaborative filtering data
train_labels, train_x, val_labels, val_x, test_labels, test_x = load_users_projects()

In [144]:
recommender = CFRecommender(k)

In [145]:
test_x.count_nonzero()

8

#### Create our user profile

In [103]:
profile_idx = 1

# Get the projects used to make the prediction
profile_col = np.squeeze(np.asarray(train_x.getcol(profile_idx).todense())).reshape(1,-1)
labels = np.asarray(train_labels.index)

In [104]:
labels[np.nonzero(profile_col)[1]]

array([  3,   8,  18,  21, 128, 133, 276, 284, 356, 458, 495])

In [105]:
# Make a prediction for 
predictions = model.predict([profile_col, labels])

In [106]:
np.count_nonzero(predictions[0])

1029

#### Get the Top-K Predictions

In [107]:
# Get the indices of all projects that have already been done by this 
done_idx = profile_col.nonzero()

In [108]:
done_idx

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([  3,   8,  18,  21, 128, 133, 276, 284, 356, 458, 495]))

In [109]:
# Set all the done projects to 0 in the predictions (so we don't pick them again)
predictions[done_idx] = 0

In [110]:
np.count_nonzero(predictions[0])

1018

In [111]:
# Get the indices of the top projects
y_pred_indices = predictions.reshape(train_labels.shape[0]).argsort()[-k:][::-1]

### Investigate the projects that were picked

In [112]:
y_pred_indices

array([228, 600, 513,  63, 796])

In [113]:
predictions[0][y_pred_indices]

array([0.07145813, 0.07141674, 0.07141015, 0.07121357, 0.07089725],
      dtype=float32)

In [114]:
project_ids = train_labels.iloc[y_pred_indices].values.flatten()
projects[projects['project_id'].isin(project_ids)]

,UN_regions,country,description,error,guid,origin,regions,tags,title,topics,url,project_id
87,[],USA,Texas Bee Watchers aims to increase awareness ...,NaN,c29812ca-a0e0-5af6-8688-132d00053a5c,scistarter,"[{'geometry': {'type': 'MultiPolygon', 'coordi...",[],"Texas Bee Watchers: 52 Gardens, 52 Weeks","[Animals, Biology, Ecology & Environment, Natu...",https://scistarter.com/project/209-Texas-Bee-W...,209
370,[],,The Tucson Bird Count is an annual project in ...,NaN,230bc5d1-e929-5aec-843f-bbe9c2ba4e7b,scistarter,"[{'geometry': {'type': 'MultiPolygon', 'coordi...","[arizona, bird, birdwatching, desert, reconcil...",Tucson Bird Count,"[Nature & Outdoors, Birds]",https://scistarter.com/project/606-Tucson-Bird...,606
839,[],USA,"FLOW, which stands for Follow and Learn about ...",NaN,ae184466-e5ea-5fb0-bbd1-0c5219cf2ab9,scistarter,"[{'geometry': {'type': 'MultiPolygon', 'coordi...","[california, education, hab, harmful algal blo...",FLOW Program,"[Science Policy, Education, Nature & Outdoors,...",https://scistarter.com/project/1316-FLOW-Progr...,1316
976,[],None,Invasive weedy plants are a widespread problem...,NaN,wilsoncenter:188-None,wilsoncenter,"[{'geometry': {'type': 'MultiPolygon', 'coordi...","[aquatic plants, Aquatic plants, invasive terr...",Invasive Plant Atlas of the MidSouth,[],https://scistarter.com/project/1694-Invasive-P...,1694
1256,[],None,The North Carolina King Tides Project is an in...,NaN,0be5ace5-81ab-5d7b-bd55-63388aab90b5,scistarter,"[{'geometry': {'type': 'MultiPolygon', 'coordi...","[aquatic, beach, beaches, climate change, coas...",North Carolina King Tides Project,"[Science Policy, Ocean, Water, Marine & Terres...",https://scistarter.com/project/17581-North-Car...,17581


In [115]:
train_x[y_pred_indices].shape

(5, 880)

In [116]:
for i in range(0, train_x[y_pred_indices].shape[0]):
    row = train_x[y_pred_indices].getrow(i)
    print(str(row.nnz))

2
2
2
2
14


In [129]:
val_x.shape

(1029, 880)

In [132]:
len(np.where(~val_x.todense().any(axis=0))[0])

879

In [130]:
test_x.shape

(1029, 880)

In [133]:
len(np.where(~test_x.todense().any(axis=0))[0])

879

So, in summary. My data validation and test data is blank for every y_pred and y_true

In [117]:
# return the project_ids of the top recommendations
recommendations = train_labels.iloc[y_pred_indices]

#### Generate the y_pred and y_true for evaluation

In [118]:
y_true, y_pred = recommender.generate_y(recommendations, train_labels, test_x.getcol(profile_idx), val_x=val_x.getcol(profile_idx))

#### Get precision and recall

In [119]:
precision, recall, fscore, support = precision_recall_fscore_support(y_true, y_pred, average='binary', pos_label=1)

/Users/thomascartwright/miniconda3/envs/cdea/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [120]:
precision

0.0

In [121]:
recall

0.0